In [3]:
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power

from math import asin
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm_notebook

In [4]:
df = pd.read_csv ('А_B_csv.csv')
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
...,...,...,...,...,...
90184,9999441,gate_40,97,1,0
90185,9999479,gate_40,30,0,0
90186,9999710,gate_30,28,1,0
90187,9999768,gate_40,51,1,0


### Я так понимаю, что 'retention_1' и 'retention_7' - это время удержания (день/неделя)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


### Пропусков нет, 1 объектовая колонка, остальные целочисленные

In [6]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [7]:
df.userid.value_counts()

116        1
6632278    1
6658202    1
6658194    1
6658134    1
          ..
3347358    1
3347337    1
3346992    1
3346979    1
9999861    1
Name: userid, Length: 90189, dtype: int64

In [8]:
control = df[df['version'] == 'gate_30']
test = df[df['version'] == 'gate_40']

In [9]:
control.shape, test.shape

((44700, 5), (45489, 5))

In [10]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=control.sum_gamerounds, name='control',))
fig.add_trace(go.Histogram(x=test.sum_gamerounds, name='test',))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [11]:
from distfit import distfit


dfit = distfit()


dfit.fit_transform(test.sum_gamerounds)

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.00669022] [loc=51.299 scale=103.293]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 0.00290985] [loc=0.000 scale=51.299]
[distfit] >INFO> [pareto    ] [0.01 sec] [RSS: 0.00027492] [loc=-19.632 scale=19.632]
[distfit] >INFO> [dweibull  ] [1.29 sec] [RSS: 0.0107812] [loc=4.000 scale=32.065]
[distfit] >INFO> [t         ] [2.78 sec] [RSS: 0.0027232] [loc=9.533 scale=10.425]
[distfit] >INFO> [genextreme] [2.91 sec] [RSS: 0.000342667] [loc=8.127 scale=12.149]
[distfit] >INFO> [gamma     ] [1.54 sec] [RSS: 0.00711806] [loc=-0.000 scale=3.949]
[distfit] >INFO> [lognorm   ] [0.29 sec] [RSS: 0.000141125] [loc=-0.262 scale=15.210]
[distfit] >INFO> [beta      ] [2.21 sec] [RSS: 0.000787604] [loc=-0.000 scale=19952.031]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.00830253] [loc=0.000 scale=2640.000]
[distfit] >INFO> [loggamma  ] [0.26 sec] [RSS: 0.00677248] [loc=-33690.830 scale=4551.557]
[distfit]

{'model': {'name': 'lognorm',
  'score': 0.00014112533231966308,
  'loc': -0.26168682590540004,
  'scale': 15.210018233618559,
  'arg': (1.7157500071954126,),
  'params': (1.7157500071954126, -0.26168682590540004, 15.210018233618559),
  'model': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1d1479a86a0>,
  'bootstrap_score': 0,
  'bootstrap_pass': None,
  'color': '#e41a1c',
  'CII_min_alpha': 0.6429666418070866,
  'CII_max_alpha': 255.46568605642997},
 'summary':           name     score          loc         scale  \
 0      lognorm  0.000141    -0.261687     15.210018   
 1       pareto  0.000275   -19.632085     19.632085   
 2   genextreme  0.000343       8.1271     12.149293   
 3         beta  0.000788         -0.0  19952.031234   
 4            t  0.002723     9.533338     10.424744   
 5        expon   0.00291          0.0     51.298776   
 6         norm   0.00669    51.298776    103.293281   
 7     loggamma  0.006772 -33690.83029   4551.556572   
 8        gam

In [12]:
from distfit import distfit


dfit = distfit()


dfit.fit_transform(control.sum_gamerounds)

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.00777477] [loc=52.456 scale=256.714]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 0.0030265] [loc=0.000 scale=52.456]
[distfit] >INFO> [pareto    ] [0.01 sec] [RSS: 0.000363798] [loc=-21.267 scale=21.267]
[distfit] >INFO> [dweibull  ] [1.26 sec] [RSS: 0.0042289] [loc=4.000 scale=57.745]
[distfit] >INFO> [t         ] [2.16 sec] [RSS: 0.00305106] [loc=10.233 scale=11.115]
[distfit] >INFO> [genextreme] [1.65 sec] [RSS: 0.0103751] [loc=2.894 scale=3.014]
[distfit] >INFO> [gamma     ] [1.40 sec] [RSS: 0.0086819] [loc=-0.000 scale=4.044]
[distfit] >INFO> [lognorm   ] [0.34 sec] [RSS: 0.000185156] [loc=-0.283 scale=15.646]
[distfit] >INFO> [beta      ] [2.27 sec] [RSS: 0.00032601] [loc=-0.000 scale=294233.073]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.00867557] [loc=0.000 scale=49854.000]
[distfit] >INFO> [loggamma  ] [0.25 sec] [RSS: 0.00819642] [loc=-145952.826 scale=19932.412]
[distfit

{'model': {'name': 'lognorm',
  'score': 0.00018515647144961954,
  'loc': -0.28316153737793265,
  'scale': 15.64574215175641,
  'arg': (1.6904554729140937,),
  'params': (1.6904554729140937, -0.28316153737793265, 15.64574215175641),
  'model': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x1d1479aa1d0>,
  'bootstrap_score': 0,
  'bootstrap_pass': None,
  'color': '#e41a1c',
  'CII_min_alpha': 0.6869414866040597,
  'CII_max_alpha': 252.05008702261313},
 'summary':           name     score            loc          scale  \
 0      lognorm  0.000185      -0.283162      15.645742   
 1         beta  0.000326      -0.000001  294233.073404   
 2       pareto  0.000364     -21.267187      21.267187   
 3        expon  0.003026            0.0      52.456264   
 4            t  0.003051      10.232733      11.115175   
 5     dweibull  0.004229            4.0      57.744863   
 6         norm  0.007775      52.456264     256.713552   
 7     loggamma  0.008196 -145952.825686   1993

In [13]:
mw_stat, pvalue = stats.mannwhitneyu(x=control.sum_gamerounds, y=test.sum_gamerounds, )
pvalue

0.05020880772044255

In [14]:
es = (test.sum_gamerounds.mean() - control.sum_gamerounds.mean()) / ((control.sum_gamerounds.std()**2 + test.sum_gamerounds.std()**2) / 2) ** 0.5

In [15]:
tt_ind_solve_power(effect_size=es, nobs1=control.shape[0], alpha=pvalue, ratio=1)

0.14368870912447107

In [16]:
lift = (test.sum_gamerounds.mean() / control.sum_gamerounds.mean()) - 1
lift * 100

-2.2065781397397344

### Отрицательный прирост. Необходимо либо перепроводить, либо продолжать сбор данных

In [17]:
n = 1000
sample_size = max(control.shape[0], test.shape[0])
sample_mean = []
for _ in tqdm_notebook(range(n)):
    sample_control = control.sum_gamerounds.sample(n=sample_size, replace=True, random_state=_)
    sample_test = test.sum_gamerounds.sample(n=sample_size, replace=True, random_state=_)
    sample_mean.append(sample_test.mean() - sample_control.mean())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
np.quantile(sample_mean, [.025, 0.5, 0.975])

array([-3.71059267, -0.98386423,  1.07189266])

In [19]:
fig = px.histogram(pd.DataFrame({'mean':sample_mean}),
                   x='mean',
                   title='sample of mean diff',
                   marginal = 'box',
                   nbins = 50)
fig.show()

In [20]:
p_1 = stats.norm.cdf(
    x = 0, 
    loc = np.mean(sample_mean), 
    scale = np.std(sample_mean)
)
p_2 = stats.norm.cdf(
    x = 0, 
    loc = -np.mean(sample_mean), 
    scale = np.std(sample_mean)
)
p_value = min(p_1, p_2) * 2
p_value

0.3799015144923291

### У меня не сошлось с Манна-Уитни...

## Посчитаем по удержанию в 1 день

In [46]:
from statsmodels.stats.weightstats import ztest as ztest
ztest(control.retention_1, test.retention_1, value=0) 

(1.7840979256519656, 0.07440775430035963)

In [47]:
n1, n2 = control.shape[0], test.shape[0]
k1_1, k2_1 = control.retention_1.sum(), test.retention_1.sum()

In [60]:
n1, n2

(44700, 45489)

In [61]:
k1_1, k2_1

(20034, 20119)

In [64]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1_1, k2_1]), 
                                                   np.array([n1, n2]))
print('Results are ','z_score = %.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score = 1.784, pvalue = 0.074


In [68]:
chisq, chi_pvalue, table = proportion.proportions_chisquare(np.array([k1_1, k2_1]), np.array([n1, n2]))
print('Results are ','chi_score = %.3f, pvalue = %.3f'%(chisq, chi_pvalue))

Results are  chi_score = 3.183, pvalue = 0.074


In [69]:
es = 2  * asin(np.sqrt(k1_1/n1)) - 2 * asin(np.sqrt(k2_1/n2))

In [70]:
#power
zt_ind_solve_power(effect_size=es, nobs1=n1, alpha=chi_pvalue, ratio=1)

0.49709241390001524

## Посчитаем по удержанию в 7 дней

In [56]:
n1, n2 = control.shape[0], test.shape[0]
k1_7, k2_7 = control.retention_7.sum(), test.retention_7.sum()

In [57]:
n1, n2

(44700, 45489)

In [58]:
k1_7, k2_7

(8502, 8279)

In [63]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1_7, k2_7]), 
                                                   np.array([n1, n2]))
print('Results are ','z_score = %.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score = 3.164, pvalue = 0.002


In [74]:
ztest(control.retention_7, test.retention_7, value=0) 

(3.1644994996802778, 0.0015534992923098888)

In [71]:
es = 2  * asin(np.sqrt(k1_7/n1)) - 2 * asin(np.sqrt(k2_7/n2))

In [72]:
#power
zt_ind_solve_power(effect_size=es, nobs1=n1, alpha=chi_pvalue, ratio=1)

0.9141029790092572

1) Время в раундах - сократилось. 
2) Удержание в 1й день - недостаточная мощность теста, либо продолжать, либо закончить тест. 
3) Удержание в течении 7 дней - значимо изменилось.

Необходимо продолжить проведение эксперимента,